In [7]:
import cv2
import numpy as np
import os
directory = os.path.join(os.path.expanduser("~"), "Desktop", "data-train")
files = os.listdir(directory)
template_path = 'C:/Users/omogo/Desktop/Screenshot 2024-01-18 sss.png'
template = cv2.imread(template_path)
check = 0
resultArr = []

regions = [
    {"name": "left-click", "position": [40, 0, 40, 160],"has_object": False},
    {"name": "right-click", "position": [150, 0, 40, 160] ,"has_object": False},
    {"name": "scroll", "position": [95, 20, 30, 67] ,"has_object": False},
    {"name": "logo", "position": [60, 135, 100, 60] ,"has_object": False},
]

for file in files:
    # Check if the file has the .jpg extension
    if file.endswith(".jpg"):
        # Read the file
        image = cv2.imread(os.path.join(directory, file))
        if image is not None:
            print(file)
            isLength = 0
            
            # Chuyển đổi ảnh sang ảnh xám
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Áp dụng bộ lọc Canny để phát hiện biên cạnh
            edges = cv2.Canny(gray, 100, 200) 
            
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                
            rgb_edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
            
            contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            has = 0
            
            max_area = 100
            max_contour = None
            
            # for contour in contours:
            #     area = cv2.contourArea(contour)
            #     if area > max_area:
            #         has += 1 
            #         break
                
            images_with_contour = []
            images_with_contour_canny = []

            for region in regions:
                # Lấy thông tin vị trí của region
                x, y, width, height = region['position']
                # Vẽ khung màu đỏ trên hình ảnh
                cv2.rectangle(rgb_edges, (x, y), (x + width, y + height), (0, 0, 255), 2)
                # cv2.rectangle(hsv, (x, y), (x + width, y + height), (0, 0, 255), 2)
                roi = image[y:y+height, x:x+width]
                images_with_contour.append(roi)
                roi_canny = cv2.Canny(roi, 100, 200)  
                cv2.rectangle(rgb_edges, (x, y), (x + width, y + height), (0, 0, 255), 2)
                images_with_contour_canny.append(roi_canny)

            cv2.imshow('Original Image', image)
            # cv2.imshow('Canny Edges', edges)
            cv2.imshow('rgb_edges Edges', rgb_edges)
            # cv2.imshow('hsv', hsv)
            # for index, img in enumerate(images_with_contour):
            #     i_str = str(index+1)  
            #     cv2.imshow("Image with contour " + i_str + " ", img)
                
            for index, img in enumerate(images_with_contour_canny):
                i_str = str(index+1)  
                cv2.imshow("images_with_contour_canny " + i_str + " ", img)
                if index == 0 :
                    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    max_length = 0
                    max_contour = None
                    for c in contours:
                        length = cv2.arcLength(c, True)
                        if length > max_length:
                            max_length = length
                            max_contour = c

                    if max_length > 200:
                        check += 1
                        resultArr.append("Chuột trái")
                   
                
                if  index == 1:
                    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    max_length = 0
                    max_contour = None
                    for c in contours:
                        length = cv2.arcLength(c, True)
                        if length > max_length:
                            max_length = length
                            max_contour = c

                    if max_length > 200:
                        check += 1
                        resultArr.append("Chuột phải")
                if index == 3:
                    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                    area = cv2.countNonZero(img)
                    if area > 100 and area < 300:
                        resultArr.append("Logo")
                        check += 1
                 
            src = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            src_e = cv2.Canny(src, 100, 200) 
            templ = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
            method = cv2.TM_CCOEFF
            threshold = 0.8
            result = cv2.matchTemplate(edges, templ,method)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            top_left = max_loc
            bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])
            color = (0, 0, 255)
            thickness = 2
            cv2.rectangle(gray, top_left, bottom_right, color, thickness)
            cv2.imshow("Result", gray)  
            top_left_array = np.array(top_left, dtype="int32")
            bottom_right_array = np.array(bottom_right, dtype="int32")
            coordinates2 = np.concatenate((top_left_array, bottom_right_array))
            point1 = (95, 20)
            point2 = (30, 67)
            point1_array = np.array(point1)
            point2_array = np.array(point2)
            coordinates1 = np.concatenate((point1_array, point2_array))
            is_in_range = np.all([coordinates2[0] >= coordinates1[0],coordinates2[1] >= coordinates1[1],coordinates2[2] >= coordinates1[2], coordinates2[3] >= coordinates1[3]])# Kiểm tra kết quả
            if is_in_range:
                check += 1
                resultArr.append('Con Lăn')
            print('Kết quả: ', resultArr)
            cv2.waitKey(0)
            resultArr.clear()
            check = 0
            cv2.destroyAllWindows()
            
        else:
            print("Không thể đọc tệp ảnh.")

images (1).jpg
Kết quả:  []
images (14).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Logo']
images (20).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Con Lăn']
images (21).jpg
Kết quả:  ['Logo']
images (22).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Logo']
images (29).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Logo', 'Con Lăn']
images (31).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Logo', 'Con Lăn']
images (33).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Logo', 'Con Lăn']
images (39).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Logo', 'Con Lăn']
images (4).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Logo', 'Con Lăn']
images (41).jpg
Kết quả:  ['Chuột trái', 'Chuột phải', 'Con Lăn']
images (7).jpg
Kết quả:  []
